# **Veri Çekme**

In [13]:
pip install --user git+https://github.com/woluxwolu/twint.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/woluxwolu/twint.git to /tmp/pip-req-build-48x7thj6
  Running command git clone -q https://github.com/woluxwolu/twint.git /tmp/pip-req-build-48x7thj6


In [1]:
#Gerekli kütüphaneleri dahil ediyoruz
import requests    
from bs4 import BeautifulSoup
import pandas as pd
import twint
import random
import nest_asyncio
nest_asyncio.apply()

In [2]:
agent={"User-Agent":# internet tarayıcınıza my user agent yazın}


In [3]:
def twitter(basliklar:list=["deneme"],limit:int=10,baslangic:str="2015-01-24 00:00:00",
                bitis:str ='2022-07-21 00:00:00',kaydet:bool=True,dosya_adi:str="-"):
   
    for isim in basliklar:
        
        t =twint.Config()
        t.Search = isim
        t.Limit =limit
        t.Since = baslangic
        t.Until = bitis
        
        if kaydet==True:
            t.Store_csv = True,
            
            if dosya_adi =="-":
                yeni_dosya_adi=isim+str(random.randint(10000,100000))+".csv"
            else:
                yeni_dosya_adi = dosya_adi+".csv"
            t.Output=yeni_dosya_adi
       
        twint.run.Search(t)

In [4]:
def turkce_harfleri_degistir(x):
   #stringdeki (ü,o,ş,ğ,ı,c) harflerini (u,o,s,g,i,c) ile değiştirecek
    x = x.replace('ü', 'u')
    x = x.replace("ö","o")
    x = x.replace("ş","s")
    x = x.replace("ğ","g")
    x = x.replace("ı","i")
    x = x.replace("ç","c")
    return x

In [5]:
def sikayet_var(isim,agent,degerleri_yazdir=True): #daha bitmedi
    
    url ="https://www.sikayetvar.com/"+isim
    r =requests.get(url,headers=agent)
    baslik=r.content
    soup =BeautifulSoup(baslik,"html.parser")
    
    basliklar =soup.find_all("a",{"class":"complaint-layer"})
    
    sikayet_basliklari =[]
    sikayet_linkleri=[]
    
    for i in basliklar:
        
        sikayet_basliklari.append(i.text)
        if degerleri_yazdir == True:
            print(i.text)
            print("\n-------------------------------------------------------------------------------------------------------------------------------------------------------")
        

        
    for l in sikayet_basliklari:
        
        l=turkce_harfleri_degistir(l)
        l=l.replace(" ","-").lower()
        link = link=url+"/"+l
        sikayet_linkleri.append(l)
        
        if degerleri_yazdir == True:
            print(link)
            print("\n-------------------------------------------------------------------------------------------------------------------------------------------------------")
            
            
    return [sikayet_basliklari],[sikayet_linkleri]

In [6]:
 def eksi_sozluk(isim:str,agent,limit=20,degerleri_yazdir=False):

        isim = turkce_harfleri_degistir(isim)
        url =f"https://eksisozluk.com/{isim}"
        r =requests.get(url,headers=agent)
        soup =BeautifulSoup(r.content,"html.parser")

        topicid=soup.find("div",{"class":"lazy"})
        sayfa_sayisi = soup.find("div",{"class":"pager"})

        topicid=str(topicid)
        sayfa_sayisi=str(sayfa_sayisi)

        topicid = ''.join(x for x in topicid if x.isdigit())
        sayfa_sayisi = ''.join(x for x in sayfa_sayisi if x.isdigit())
        sayfa_sayisi=(sayfa_sayisi[1:])

    
    
        isim = isim.replace(" ","-")

        yazilar=[]

        if (sayfa_sayisi or topicid)== "":
            print("Böyle bir entry bulunamadı ")
            return 0 

        else:

            for i in range(1,int(sayfa_sayisi)):
                if i>limit :
                    break

                else:
                    url =f"https://eksisozluk.com/{isim}--{topicid}?p={i}"
                    if degerleri_yazdir ==True:
                        print(f"url:\n{url}")
                    r =requests.get(url,headers=agent)

                    soup =BeautifulSoup(r.content,"html.parser")

                    yorumlar =soup.find_all("div",{"class":"content"})

                    for y in yorumlar:
                        yazilar.append(y.text)
                        if degerleri_yazdir ==True:
                            print(y.text)
                            
            return pd.DataFrame(yazilar,columns =['Yorum'])

In [7]:
def hepsi_burada(link,agent,limit=20,degerleri_yazdir=False,kaydet=True):
    url = f"https://www.hepsiburada.com/{link}-yorumlari"
    r = requests.get(url,headers=agent)
    soup = BeautifulSoup(r.content,"html.parser")
   
    sayfa_sayisi=soup.find_all("span",{"class":"hermes-PageHolder-module-1QoWq"})
    sayfa_sayisi = str(sayfa_sayisi[-1].text)
    yazilar=[]

    for i in range(1,int(sayfa_sayisi)//2):#hepsi buradada toplam yorum sayısının yarısından fazlasını almaya çalıştığınızda ilk yorumlar sayfasına gönderiliyorsunuz.
        if i>limit:
            break

        else:
            yeni_url=url+"?sayfa="+str(i)
            r = requests.get(yeni_url,headers=agent)
            soup = BeautifulSoup(r.content,"html.parser")
            yorumlar =soup.find_all("div",{"class":"hermes-ReviewCard-module-2dVP9"})

            for a in yorumlar:
                yorum =a.text.split("Değerlendirilen özellikler")
                yorum=str(yorum[0])
                yazilar.append(yorum)
                if degerleri_yazdir ==True:
                    print(yorum+"\n")

            if degerleri_yazdir ==True:
                print("----------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        
    return pd.DataFrame(yazilar,columns =['Yorum'])


In [8]:
ek1 =eksi_sozluk(isim="turkcell",agent=agent,limit=100,degerleri_yazdir=False)
ek1.to_csv ('ek1.csv',index =False)

ek2 = eksi_sozluk(isim="turk telekom",agent=agent,limit=100,degerleri_yazdir=False)
ek2.to_csv ('ek2.csv',index =False)

In [9]:
hb1 = hepsi_burada(link= "samsung-galaxy-buds-pro-bluetooth-kulaklik-phantom-white-sm-r190nzwatur-p-HBCV00001UKZXS",
             agent=agent,limit=65,degerleri_yazdir=False)
hb1.to_csv ('hb1.csv',index =False)

In [10]:
hb2=hepsi_burada(link="boya-profesyonel-youtuber-yaka-mikrofonu-by-m1-mikrofon-p-HBV00000EDMPB",agent=agent,limit=65,degerleri_yazdir=False)
hb1.to_csv ('hb2.csv',index =False)

In [11]:
basliklar =["fenerbahce","TOGG","yarısma","teknofest","turk telekom"]
twitter(basliklar,limit=200,dosya_adi="tweetler")

1549906926854619136 2022-07-20 23:59:41 +0000 <lazkopatramo> @ResulDurmus61 İçimizdeki hayin senmiydin lan 2. Resıl seni esefle kınıyorum bilsemki Fenerbahçe yenince biz avrupa şampiyonu olcak yine desteklemem olum az akıllı lan onlar bizi destekliyormu 😡
1549906912912752641 2022-07-20 23:59:38 +0000 <OnurTNS> @ismailyuksekk @Fenerbahce İlk geldiğin günden beri senin kadar heyecanlıyım bende inşallah daha iyi yerlere geleceksin o formayı daha fazla terleteceksin o sahanın içinde senin gibi Fenerbahçe'ye aşık olan sahada bile bir taraftar olabilen oyunculara ihtiyacımız var Allah yolunu açık etsin
1549906808713641985 2022-07-20 23:59:13 +0000 <onurlu1i> Sol bek ve defans acilllll alabilirmiyiz canoo @Fenerbahce
1549906706259386368 2022-07-20 23:58:49 +0000 <DoganCaanBektas> Fenerbahçe işte bu kadar önemli...
1549906367548473344 2022-07-20 23:57:28 +0000 <BaygulAleyna> @Fenerbahce Basarilar
1549906361697329152 2022-07-20 23:57:27 +0000 <NoyanSahh> @ufuk09894442 @Fenerbahce Yenemezdin 3 y